# GIST Driver
##Test on events of interest:
  
  texnet2024shcj

  
Assumes InjectionSQLScheduled completed

#Configuration

##Libraries

In [0]:
%restart_python

In [0]:

%run "/Workspace/_utils/Utility_Functions"

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install geopandas
!pip install geodatasets
!pip install contextily
#! pip install folium matplotlib mapclassify contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 157.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
import sys
homePath='/Workspace/Users/bill.curry@exxonmobil.com'
gistPath=homePath+'/GIST'
sys.path.append(gistPath+'/lib')
#import dataBricksConfig as db
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("eqSQL").getOrCreate()

In [0]:
import numpy as np
import pandas as pd
import os
import gistMC as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
#import contextily as cx

In [0]:
import gistPlots as gp

## Parameters

In [0]:
eventID='texnet2024shcj'

### Input Files
Produced from injectionSQLScheduled.py

In [0]:
injPath=homePath+'/injection/WeeklyRun/ScheduledOutput'

DeepWellFile=injPath+'/deep.csv'
DeepInjFile=injPath+'/deepReg.csv'
ShallowWellFile=injPath+'/shallow.csv'
ShallowInjFile=injPath+'/shallowReg.csv'
runPath=gistPath+'/runs/NorthTarzan/Outputs/'+eventID+'/'
os.makedirs(runPath, exist_ok=True)
################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runPath+'MC'

###General

In [0]:
######################
# General Parameters #
######################
nRealizations=500
nTimeBins=21
WaterDensityMin=1015.
WaterDensityMax=1025.
WaterViscosityMin=0.000799
WaterViscosityMax=0.000801
VerticalCompressibilityMin=0.00000000107
VerticalCompressibilityMax=0.00000000109
FluidCompressibilityMin=0.000000000359
FluidCompressibilityMax=0.000000000361
FaultFrictionCoeffMin=0.55
FaultFrictionCoeffMax=0.65
RockFrictionCoeffMin=0.55
RockFrictionCoeffMax=0.65

###Shallow interval

In [0]:
###############################
# Shallow Interval Parameters #
###############################
ShallowPorosityPercentMin=1.
ShallowPorosityPercentMax=20.

ShallowPermMDMin=5.
ShallowPermMDMax=200.

ShallowThicknessFTMin=200.
ShallowThicknessFTMax=2000.
# Roughly double of shallow - 
ShallowShearModulusMin=4e9
ShallowShearModulusMax=6e9
ShallowPoissonsRatioDrainedMin=0.295
ShallowPoissonsRatioDrainedMax=0.305
ShallowPoissonsRatioUndrainedMin=0.305
ShallowPoissonsRatioUndrainedMax=0.315
ShallowBiotsCoefficientMin=0.26
ShallowBiotsCoefficientMax=0.36

### Deep Interval

In [0]:
###############################
# Deep Interval Parameters #
###############################
DeepPorosityPercentMin=3.
DeepPorosityPercentMax=15.
DeepPermMDMin=5.
DeepPermMDMax=500.
# Now let's change this to something where 1D is the high range and 10mD is the low range

DeepThicknessFTMin=200.
DeepThicknessFTMax=2000.
# Roughly double of shallow - 
DeepShearModulusMin=4e9
DeepShearModulusMax=6e9
DeepPoissonsRatioDrainedMin=0.295
DeepPoissonsRatioDrainedMax=0.305
DeepPoissonsRatioUndrainedMin=0.305
DeepPoissonsRatioUndrainedMax=0.315
DeepBiotsCoefficientMin=0.26
DeepBiotsCoefficientMax=0.36


#Earthquake Selection

In [0]:
eqs=es.eqSQL()
EQDF=eqs.getEarthquake(eventID)
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# What about the other fault plane solution? - 251 / 36 / -97
EQDF['Strike']=80.
EQDF['Dip']=55.
EQDF['Rake']=-85.
# We could make use of the Rake
EQDF.info()
eq=EQDF.loc[0]
EQDF.to_csv(runPath+'/EQ.csv')

getEarthquake:     SeismicEventId  ... B3RecordDeletedUTCDateTime
0         2496144  ...                        NaT

[1 rows x 32 columns]
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              1 non-null      int64         
 1   DataSource                  1 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     1 non-null      object        
 4   EventTimeUtc                1 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    1 non-null      object        
 6   EventTimeZone               1 non-null      object        
 7   EventType                   1 non-null      object        
 8   DepthKm                     1 non-null      float64       
 9   DepthErrorKm                1 non-null     

#GIST Initialization

## Shallow 

In [0]:
# Initialize GIST instance for Shallow interval #
##################################################
shallowM=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
##################################################################
# Initialize pore pressure model                                 #
##################################################################
shallowM.initPP(rho0_min=WaterDensityMin,
             rho0_max=WaterDensityMax,
             phi_min=ShallowPorosityPercentMin,
             phi_max=ShallowPorosityPercentMax,
             kMD_min=ShallowPermMDMin,
             kMD_max=ShallowPermMDMax,
             h_min=ShallowThicknessFTMin,
             h_max=ShallowThicknessFTMax,
             alphav_min=VerticalCompressibilityMin,
             alphav_max=VerticalCompressibilityMax,
             beta_min=FluidCompressibilityMin,
             beta_max=FluidCompressibilityMax)

In [0]:
# These parameters need more vetting #
######################################
shallowM.initPE(mu_min=ShallowShearModulusMin,
             mu_max=ShallowShearModulusMax,
             nu_min=ShallowPoissonsRatioDrainedMin,
             nu_max=ShallowPoissonsRatioDrainedMax,
             nu_u_min=ShallowPoissonsRatioUndrainedMin,
             nu_u_max=ShallowPoissonsRatioUndrainedMax,
             alpha_min=ShallowBiotsCoefficientMin,
             alpha_max=ShallowBiotsCoefficientMax,
             mu_f_min=FaultFrictionCoeffMin,
             mu_f_max=FaultFrictionCoeffMax,
             mu_r_min=RockFrictionCoeffMin,
             mu_r_max=RockFrictionCoeffMax,
             match=True)
shallowM.writeRealizations(runPath+'/shallowRealizations.csv')

## Deep

In [0]:
# Initialize GIST instance for deep interval #
##################################################
deepM=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
##################################################################
# Initialize pore pressure model                                 #
##################################################################
deepM.initPP(rho0_min=WaterDensityMin,
             rho0_max=WaterDensityMax,
             phi_min=DeepPorosityPercentMin,
             phi_max=DeepPorosityPercentMax,
             kMD_min=DeepPermMDMin,
             kMD_max=DeepPermMDMax,
             h_min=DeepThicknessFTMin,
             h_max=DeepThicknessFTMax,
             alphav_min=VerticalCompressibilityMin,
             alphav_max=VerticalCompressibilityMax,
             beta_min=FluidCompressibilityMin,
             beta_max=FluidCompressibilityMax)

# Write histogram of parameters to csv file:


In [0]:
# These parameters need more vetting #
######################################
deepM.initPE(mu_min=DeepShearModulusMin,
             mu_max=DeepShearModulusMax,
             nu_min=DeepPoissonsRatioDrainedMin,
             nu_max=DeepPoissonsRatioDrainedMax,
             nu_u_min=DeepPoissonsRatioUndrainedMin,
             nu_u_max=DeepPoissonsRatioUndrainedMax,
             alpha_min=DeepBiotsCoefficientMin,
             alpha_max=DeepBiotsCoefficientMax,
             mu_f_min=FaultFrictionCoeffMin,
             mu_f_max=FaultFrictionCoeffMax,
             mu_r_min=RockFrictionCoeffMin,
             mu_r_max=RockFrictionCoeffMax,
             match=True)

deepM.writeRealizations(runPath+'/deepRealizations.csv')

#Well selection

####Input wells

In [0]:
#########################################
# Add potential wells to each instance  #
#########################################
print("Before adding shallow wells")
shallowM.addWells(ShallowWellFile,ShallowInjFile,verbose=1)

Before adding shallow wells
 gistMC.addWells: well file added with  7977  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'UniqueWellIdentifier', 'UICNumber',
       'APINumber', 'LeaseName', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType', 'WellName',
       'WellNumber', 'State', 'Basin', 'County', 'District', 'SRAOrSIR',
       'B3InjectionType', 'B3InjectionStatus', 'RegulatoryInjectionType',
       'PermittedMaxLiquidBPD', 'PermittedMaxLiquidPSIG',
       'PermittedMaxGasMCFPerDay', 'PermittedMaxGasPSIG',
       'PermittedCommercialStatus', 'PermittedIntervalTopFt',
       'PermittedIntervalBottomFt', 'InjectionClass', 'PermitStage',
       'PermittedWellDepthClassification', 'DaysApplicationHasBeenInReview',
       'DaysToPermitApproval', 'PermitIsAmendment',
       'PendingApplicationIsAmendment',
       'PendingApplicationRequestedCommercialStatus',
       'PendingApplicationRequestedIntervalTopFt',
       'Pen

In [0]:
print("Before adding deep wells")
deepM.addWells(DeepWellFile,DeepInjFile,verbose=1)

Before adding deep wells
 gistMC.addWells: well file added with  2836  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'UniqueWellIdentifier', 'UICNumber',
       'APINumber', 'LeaseName', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType', 'WellName',
       'WellNumber', 'State', 'Basin', 'County', 'District', 'SRAOrSIR',
       'B3InjectionType', 'B3InjectionStatus', 'RegulatoryInjectionType',
       'PermittedMaxLiquidBPD', 'PermittedMaxLiquidPSIG',
       'PermittedMaxGasMCFPerDay', 'PermittedMaxGasPSIG',
       'PermittedCommercialStatus', 'PermittedIntervalTopFt',
       'PermittedIntervalBottomFt', 'InjectionClass', 'PermitStage',
       'PermittedWellDepthClassification', 'DaysApplicationHasBeenInReview',
       'DaysToPermitApproval', 'PermitIsAmendment',
       'PendingApplicationIsAmendment',
       'PendingApplicationRequestedCommercialStatus',
       'PendingApplicationRequestedIntervalTopFt',
       'Pendin

In [0]:
shallowM.wellDF.info()
EQDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7977 entries, 0 to 7976
Data columns (total 61 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   ID                                                       7977 non-null   int64  
 1   InjectionWellId                                          7977 non-null   int64  
 2   UniqueWellIdentifier                                     7977 non-null   int64  
 3   UICNumber                                                4701 non-null   object 
 4   APINumber                                                7977 non-null   object 
 5   LeaseName                                                7758 non-null   object 
 6   Operator                                                 7976 non-null   object 
 7   OperatorType                                             4360 non-null   object 
 8   OperatorPrincipalCompany    

In [0]:
deepPPSelectedWells,deepPPIgnoredWells,deepPPInj=deepM.findWells(eq,PE=False)
#deepPESelectedWells,deepPEIgnoredWells,deepPEInj=deepM.findWells(eq,PE=True)

In [0]:
minYear=-40
diffRange=(min(deepM.diffPPVec),max(deepM.diffPPVec))
deepRTDF,deepWellsDF = gi.prepRTPlot(deepPPSelectedWells,deepPPIgnoredWells,minYear,diffRange,clipYear=False)
deepWellsDF.to_csv(runPath+'/NorthTarzan_09-24_DeepPPWells.csv')
deepRTDF.to_csv(runPath+'/NorthTarzan_09-24_DeepRTDF.csv')

In [0]:
# Dump injection data to file:
deepPPInj.to_csv(runPath+'/NorthTarzan_09-24_DeepPPInj.csv')

In [0]:
eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])

/home/spark-b6222526-f5b1-46dd-835a-dc/.ipykernel/103018/command-3703220800531525-1084520577:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])


In [0]:
# Shallow well selection:
shallowPPSelectedWells,shallowPPIgnoredWells,shallowPPInj=shallowM.findWells(eq,PE=False)
#shallowPESelectedWells,shallowPEIgnoredWells,shallowPEInj=shallowM.findWells(eq,PE=True)

sDiffRange=(min(shallowM.diffPPVec),max(shallowM.diffPPVec))
shallowRTDF,shallowWellsDF = gi.prepRTPlot(shallowPPSelectedWells,shallowPPIgnoredWells,minYear,sDiffRange,clipYear=False)
shallowWellsDF.to_csv(runPath+'/NorthTarzan_09-24_ShallowPPWells.csv')
shallowRTDF.to_csv(runPath+'/NorthTarzan_09-24_ShallowRTDF.csv')
shallowPPInj.to_csv(runPath+'/NorthTarzan_09-24_ShallowPPInj.csv')

#Initial GIST runs


In [0]:
# Run deep scenarios
dWellPPScenarios=deepM.runPressureScenarios(eq,deepPPSelectedWells,deepPPInj,verbose=1)
dWellPPScenarios.to_csv(runPath+'deepPP.csv')
smallDPPDF,smallDWellList=gi.summarizePPResults(dWellPPScenarios,deepPPSelectedWells,threshold=0.5,nOrder=20,verbose=0)
smallDPPDF.to_csv(runPath+'/NorthTarzan_09-24_deepPPSubset.csv')

 gistMC.runPressureScenarios: Number of wells considered:  538
 gistMC.pressureScenario: well  0  of  538 :  1002717
 gistMC.pressureScenario: well  10  of  538 :  1003771
 gistMC.pressureScenario: well  20  of  538 :  1005818
 gistMC.pressureScenario: well  30  of  538 :  1002706
 gistMC.pressureScenario: well  40  of  538 :  1003488
 gistMC.pressureScenario: well  50  of  538 :  1004958
 gistMC.pressureScenario: well  60  of  538 :  1003176
 gistMC.pressureScenario: well  70  of  538 :  1003047
 gistMC.pressureScenario: well  80  of  538 :  1003962
 gistMC.pressureScenario: well  90  of  538 :  1003055
 gistMC.pressureScenario: well  100  of  538 :  1003122
 gistMC.pressureScenario: well  110  of  538 :  1003865
 gistMC.pressureScenario: well  120  of  538 :  1003759
 gistMC.pressureScenario: well  130  of  538 :  1003841
 gistMC.pressureScenario: well  140  of  538 :  1004594
 gistMC.pressureScenario: well  150  of  538 :  1005259
 gistMC.pressureScenario: well  160  of  538 :  1007

In [0]:

pd.Series(data=smallDWellList).to_csv(runPath+'/NorthTarzan_09-24_deepWellOrder.csv')

In [0]:
# Run shallow scenarios
sWellPPScenarios=shallowM.runPressureScenarios(eq,shallowPPSelectedWells,shallowPPInj,verbose=1)
sWellPPScenarios.to_csv(runPath+'shallowPP.csv')
smallSPPDF,smallSWellList=gi.summarizePPResults(sWellPPScenarios,shallowPPSelectedWells,threshold=0.5,nOrder=20,verbose=0)
smallSPPDF.to_csv(runPath+'/NorthTarzan_09-24_shallowPPSubset.csv')
pd.Series(data=smallDWellList).to_csv(runPath+'/NorthTarzan_09-24_shallowWellOrder.csv')

 gistMC.runPressureScenarios: Number of wells considered:  210
 gistMC.pressureScenario: well  0  of  210 :  2009719
 gistMC.pressureScenario: well  10  of  210 :  2041096
 gistMC.pressureScenario: well  20  of  210 :  2050698
 gistMC.pressureScenario: well  30  of  210 :  2058375
 gistMC.pressureScenario: well  40  of  210 :  2072223
 gistMC.pressureScenario: well  50  of  210 :  2091943
 gistMC.pressureScenario: well  60  of  210 :  2100833
 gistMC.pressureScenario: well  70  of  210 :  2108473
 gistMC.pressureScenario: well  80  of  210 :  2120269
 gistMC.pressureScenario: well  90  of  210 :  2079697
 gistMC.pressureScenario: well  100  of  210 :  2119515
 gistMC.pressureScenario: well  110  of  210 :  2108813
 gistMC.pressureScenario: well  120  of  210 :  2043375
 gistMC.pressureScenario: well  130  of  210 :  2104090
 gistMC.pressureScenario: well  140  of  210 :  2120986
 gistMC.pressureScenario: well  150  of  210 :  2103893
 gistMC.pressureScenario: well  160  of  210 :  2126

In [0]:
# Get shortened list of wells to output time series for
orderedWellList = dWellPPScenarios.groupby('ID')['Pressures'].max().sort_values(ascending=False)
print(orderedWellList)
threshold=1.
winWellList=orderedWellList[orderedWellList>1.]
print(winWellList)


ID
2113854    44.377284
2108688    18.489854
2081404    15.969688
2119704    14.024725
2112544    13.564338
             ...    
1004555     0.000000
1004553     0.000000
1004552     0.000000
1004551     0.000000
2128703     0.000000
Name: Pressures, Length: 538, dtype: float64
ID
2113854    44.377284
2108688    18.489854
2081404    15.969688
2119704    14.024725
2112544    13.564338
2120624    13.249530
2113855     8.070913
2110718     7.743706
2127004     6.378711
2109578     5.149343
2119988     4.868542
2048738     4.795006
2112307     4.537593
2059905     4.228660
2096222     4.089169
2087824     3.390969
2121195     3.332532
2015612     3.288915
2086897     3.200349
2120458     3.153430
2094440     2.783506
2119505     2.777321
2121605     2.173195
2121818     2.113527
2121764     1.579934
2123921     1.502502
2120101     1.261374
2116374     1.199011
2121854     1.167559
2015971     1.129553
2111821     1.105421
Name: Pressures, dtype: float64


In [0]:
# This is too slow! - first take top X wells contributing and then run it - maybe top 20?
winDeepPPSelectedWells=deepPPSelectedWells[deepPPSelectedWells['ID'].isin(winWellList.index)].reset_index()
print(winDeepPPSelectedWells)
winDeepPPSelectedInj=deepPPInj[deepPPInj['ID'].isin(winWellList.index)]
dWellPPScenariosF,dPDeep,dWellIDs,dayVecD = deepM.runPressureScenariosTimeSeries(eq,winDeepPPSelectedWells,winDeepPPSelectedInj,verbose=2)

    index       ID  ...    TotalBBL  Selected
0     247  2015971  ...  33743122.0      True
1     270  2048738  ...  86503043.0      True
2     273  2059905  ...  78046992.0      True
3     304  2094440  ...  60699050.0      True
4     307  2096222  ...   9888305.0      True
5     314  2113855  ...  20928513.0      True
6     316  2119988  ...  11239732.0      True
7     318  2120101  ...  42240453.0      True
8     320  2120624  ...  14467447.0      True
9     326  2121195  ...  35246775.0      True
10    334  2121854  ...  12937163.0      True
11    335  2123921  ...   7550012.0      True
12    342  2015612  ...  56567466.0      True
13    345  2109578  ...  14815678.0      True
14    346  2110718  ...  25291884.0      True
15    348  2116374  ...   8491376.0      True
16    349  2112307  ...  13310313.0      True
17    351  2112544  ...   8283102.0      True
18    352  2119505  ...  13721157.0      True
19    355  2121605  ...  13376296.0      True
20    356  2121764  ...  15623304.

In [0]:
# This runs out of memory
dWellPPScenariosF2,dPDeepF2,wellIDs1A,dayVec1A = deepM.runPressureScenariosTimeSeriesTest(eq,winDeepPPSelectedWells,winDeepPPSelectedInj,verbose=2)
dWellPPScenariosF2.to_csv(runPath+'deepPPF2.csv')

 prepInj: first day  5520.0  with interval of  10.0  days
 prepInj: min,max indicies  1 1451
runPressureScenariosTimeSeries input time axis information - nt: 1453 ; ot: 5520.0 ; dt: 10.0  earthquake index:  1446  f  0.2999999999999545
runPressureScenariosTimeSeries output time axis information - len(secArray): 1453 ; min(secArray): 5520.0 ; dt: 864000.0  earthquake index:  1446  f  0.2999999999999545
runPressureScenariosTimeSeries r2 min/max:  16329987.31551469 900683280.998593
runPressureScenariosTimeSeries TSOver4TT min/max:  0.5247810359721807 47.152088512471735
runPressureScenariosTimeSeries ppp min/max:  8569667.66084837 42469097787.349106
runPressureScenariosTimeSeries durations min/max:  864000.0 1255392000.0
runPressureScenariosTimeSeries epp min/max:  0.0 4.416573192345252
runPressureScenariosTimeSeriesTest Well  1  of  31
runPressureScenariosTimeSeriesTest Well  11  of  31
runPressureScenariosTimeSeriesTest Well  21  of  31
runPressureScenariosTimeSeriesTest Well  31  of  31


In [0]:
dPPTimeSeriesSum=dPDeep.sum(axis=1)

In [0]:
np.savez_compressed(runPath+'deepDeltaPPSum.npz', deltaPP=dPDeep,dayVec=dayVecD,wellIDs=dWellIDs)

In [0]:
np.savez_compressed(runPath+'deepDeltaPPSumC.npz', deltaPP=dPDeepF2,dayVec=dayVec1A,wellIDs=wellIDs1A)

In [0]:
# Window out significant wells - save individual time series for those


In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()

In [0]:
# dPPTimeSeries is time, well, realization - need top wells with time series of pressures
dPPWellIndices=deepPPWells['ID'].isin(deepWindowedWells)
print(sum(dPPWellIndices),' wells out of ',len(dPPWellIndices),' have >0.1 PSI contribution')
dPPTimeSeriesSelected=dPDeep[:,dPPWellIndices,:]
dPPSelectedWellNameDF=deepPPWells['WellName'][dPPWellIndices]
print(dPPSelectedWellNameDF.to_list())

In [0]:
i=0
for wellName in dPPSelectedWellNameDF.to_list():
  np.save(runPath+'deepPPTimeSeries'+wellName+'.npy',dPPTimeSeriesSelected[:,i,:])
  i=i+1
  print('Well '+wellName+' written '+i)
dPPSelectedWellNameDF.to_csv(runPath+'deepSelectedWellNames.csv')

In [0]:
dWellPEScenarios=deepM.runPoroelasticScenarios(eq,deepPEWells,deepPEInj,verbose=1)

In [0]:
sWellPEScenarios=shallowM.runPoroelasticScenarios(eq,shallowPEWells,shallowPEInj,verbose=1)

In [0]:

dWellPEScenarios.to_csv(runPath+'deepPE.csv')

In [0]:

sWellPEScenarios[sWellPEScenarios['Realization']<250].to_csv(baseOutPath+'shallowPE0.csv')
sWellPEScenarios[sWellPEScenarios['Realization']>250].to_csv(baseOutPath+'shallowPE1.csv')


In [0]:
sWellPEScenarios.info()

In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
maxPressureList=[]
for ID in deepWindowedWells:
  maxPressure=max(filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtDWellPPScenarios['MaxPressure']=maxPressureList
filtDWellPPScenarios=filtDWellPPScenarios.sort_values(by='MaxPressure',ascending=False)
shallowWindowedWells=sWellPPScenarios[sWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtSWellPPScenarios=sWellPPScenarios[sWellPPScenarios['ID'].isin(shallowWindowedWells)]
maxPressureList=[]
for ID in shallowWindowedWells:
  maxPressure=max(filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtSWellPPScenarios['MaxPressure']=maxPressureList
filtSWellPPScenarios=filtSWellPPScenarios.sort_values(by='MaxPressure',ascending=False)

In [0]:
filtDWellPPScenarios=filtDWellPPScenarios.merge(deepPPWells,on='ID')
filtSWellPPScenarios=filtSWellPPScenarios.merge(shallowPPWells,on='ID')
print('deep ',filtDWellPPScenarios.shape[0],', shallow',filtSWellPPScenarios.shape[0])

## Save Results

In [0]:
# Save results - visualize separately!
baseOutPath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/Stanton11-4-23_v2_'
#dWellPPScenarios.to_csv(baseOutPath+'deepPP.csv')
dWellPEScenarios.to_csv(baseOutPath+'deepPE.csv')
#sWellPPScenarios.to_csv(baseOutPath+'shallowPP.csv')
sWellPEScenarios.to_csv(baseOutPath+'shallowPE.csv')
#np.save(baseOutPath+'deepPPTimeSeries0.csv',dPPTimeSeries['Realization'<100])
#np.save(baseOutPath+'shallowPPTimeSeries.csv',sPPTimeSeries)

## Sanity check of initial results
### Check injection on wells with non-negligable pressure contributions

In [0]:
# Select injection histories of wells
deepWindowedInjection=deepPPInj[deepPPInj['ID'].is_in(deepWindowedWells)]
shallowWindowedInjection=shallowPPInj[shallowPPInj['ID'].is_in(shallowWindowedWells)]
deepInjQC=deepWindowedInjection.merge(deepPPWells[['Name','Operator','APINumber','Latitude','Longitude']],on='ID',how='left')
shallowInjQC=shallowWindowedInjection.merge(shallowPPWells[['Name','Operator','APINumber','Latitude','Longitude']],on='ID',how='left')

In [0]:
# Make stack plots of recent injection - Deep
injQC=deepInjQC
injQC['Date']=pd.to_datetime(injQC.Date)
# Generate x and y
dates=injQC['Date'].drop_duplicates()
# Loop over wells
oDate=min(dates)
eDate=max(dates)
dateRange=pd.date_range(start=oDate,end=eDate, freq='10D')
# Get x min, x max
wellIDs=injQC['ID'].drop_duplicates()
allWellBPDs=[]
allWellLabels=[]
for well in wellIDs:
  wellDates=injQC[injQC['ID']==well]['Date']
  wellBPDs=injQC[injQC['ID']==well]['BPD']
  minWD=min(wellDates)
  maxWD=max(wellDates)
  nPrepend=sum(dateRange<minWD)
  nAppend=sum(dateRange>maxWD)
  #print('Well ',well,' ',minWD,maxWD,oDate,eDate,nPrepend,' prePad, ',nAppend,' postPad, ',len(dateRange))
  #print('allWellBPDs: ',allWellBPDs)
  if nPrepend>0:
    prependZeroBPDs=pd.Series(np.zeros([nPrepend]))
  else:
    prependZeroBPDs=None
  if nAppend>0:
    appendZeroBPDs=pd.Series(np.zeros([nAppend,1]))
  else:
    appendZeroBPDs=None
  padWellBPDs=pd.concat([prependZeroBPDs,wellBPDs,appendZeroBPDs])
  allWellBPDs.append(padWellBPDs)
  #allWellLabels.append(injQC['WellName'][injQC['ID']==well][0])
  allWellLabels.append(injQC['WellName'][injQC['ID']==well].loc[0])
# Now have allWellBPDs, allWellLabels, and dateRange all set
# Generate Stacked area plot of injection
# y needs to be defined for each well, and extended for the full time series
col = sns.color_palette("hls", len(wellIDs))
#zoomDate=oneEQ['Origin Date'] - pd.DateOffset(years=2)
ax.stackplot(dateRange,allWellBPDs, labels=allWellLabels,colors=col)
ax.set_title('Selected well injection')
ax.set_xlabel('Date')
ax.set_ylabel('Rate (BPD)')
#if startDate!=None or endDate!=None: axes.xlim(startDate,endDate)
ax.legend(loc='upper left')
plt.show()

In [0]:
# Repeat for Shallow

In [0]:
# Plot Time Series of total pressures and injection histories


In [0]:

print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:

print(shallowWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:
# Concatenate pore pressure and poroelastic results
dWellPPScenarios['Physics']='Pore Pressure'
dWellPEScenarios['Physics']='Poroelastic'
dWellPPScenarios['Stresses']=dWellPPScenarios['Pressures']
allDeep=pd.concat(objs=[dWellPPScenarios,dWellPEScenarios[dWellPEScenarios['ID'].isin(dWellPPScenarios['ID'])]])
allDeep=allDeep.merge(deepPEWells,on='ID')



In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:
maxPressureList=[]
for ID in deepWindowedWells:
  maxPressure=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures']
  maxPressureList.append
dWellPPScenarios['MaxPressure']=dWellPPScenarios['ID'].map(maxPressureDict)
dWellPPScenarios=dWellPPScenarios.sort_values(by='MaxPressure',ascending=False)

In [0]:
# Select wells that only have greater than a 0.1 psi stress increase
deepWindowedWells=allDeep[allDeep['Stresses']>0.1]['ID'].unique()
allDeepWindowed=allDeep[allDeep['ID'].isin(deepWindowedWells)]
# Get maximum pressure or stress for each well, sort by that
maxList=[]
for ID in deepWindowedWells:
  # get maximum row
  maxRow=allDeepWindowed[allDeepWindowed['ID']==ID]['Stresses'].idxmax()
  maxList.append([allDeepWindowed.iloc[maxRow]['ID'],allDeepWindowed.iloc[maxRow]['Stresses']])
wellOrdered=pd.DataFrame(maxList,columns=['ID','Stresses']).sort_values(by='Stresses')
IDOrder=wellOrdered['ID'].to_list()
sortedDeepWindowed=allDeepWindowed.sort_values(by="ID", key=lambda column: column.map(lambda e: IDOrder.index(e)), inplace=True)

In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(30,200))
# Relative contributions
sns.stripplot(data=allDeep,x='Stresses',y='WellName',hue='Physics',dodge=True,jitter=True,alpha=0.1,ax=axes)
axes.set_title('Deep Well Stresses')
axes.set_xlabel('Stress (PSI)')

In [0]:
# Concatenate pore pressure and poroelastic results
sWellPPScenarios['Physics']='Pore Pressure'
sWellPEScenarios['Physics']='Poroelastic'
sWellPPScenarios['Stresses']=sWellPPScenarios['Pressures']
allShallow=pd.concat(objs=[sWellPPScenarios,sWellPEScenarios[sWellPEScenarios['ID'].isin(sWellPPScenarios['ID'])]])
allShallow=allShallow.merge(shallowPEWells,on='ID')


In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(30,10))
# Relative contributions
sns.stripplot(data=allShallow,x='Stresses',y='WellName',hue='Physics',dodge=True,jitter=True,alpha=0.1,ax=axes)
axes.set_title('Shallow Well Stresses')
axes.set_xlabel('Stress (PSI)')

In [0]:
# Plot results
fig,axes=plt.subplots(3,1, sharex=True, figsize=(60,20))
gp.disaggregationPlot(axes[0],dWellPPScenarios,dWellPEScenarios,deepPEWells,'Deep')
gp.disaggregationPlot(axes[1],sWellPPScenarios,sWellPEScenarios,shallowPEWells,'Shallow')
# Concatenate pore pressure and poroelastic results


In [0]:
# 

dayDVec=np.linspace(deepM.injOT, deepM.injOT+deepM.injDT*(deepM.injNT-1.),deepM.injNT)
dPPTimeSeriesSum=np.sum(dPPTimeSeries,axis=1)
realization=np.linspace(0,499,500)
nwC=deepPPWells.shape[0]
dTPPDF=pd.DataFrame(columns=['Days','Realization','TotalPressure'])
dPPDF=pd.DataFrame(columns=['Days','Realization','ID','Pressure','TotalPressure'])
dTPPDF['Days']=np.tile(dayDVec,deepM.nReal)
dTPPDF['TotalPressure']=np.reshape(dPPTimeSeriesSum[:,:],deepM.injNT*deepM.nReal,order='F')
dTPPDF['Realization']=np.tile(realization,deepM.injNT)
dTPPDF=gi.getDates(dTPPDF,deepM.epoch)

In [0]:
fig,axes=plt.subplots(2,1, sharex=True, figsize=(40,15))

sns.lineplot(ax=axes[0],x='Date',y='TotalPressure',hue='Realization',data=dTPPDF)
sns.scatterplot(data=filtEQs,ax=axes[0],x='Date',y='Magnitude',s=300,marker='*')
axes[0].set_title('Deep Pressure Histories')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Pressure (PSI)')
sns.lineplot(ax=axes[1],x='Date',y='TotalPressure',hue='Realization',data=sTPPDF)
sns.scatterplot(data=filtEQs,ax=axes[1],x='Date',y='Magnitude',s=300,marker='*')
plt.show()